# Fine-tuning with KerasNLP and Bias-Aware Techniques

This notebook uses the KerasNLP library to fine-tune a DeBERTa model, incorporating the bias-aware data augmentation techniques.

## 1. Install Libraries

In [1]:
# Set Keras 3 backend
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

## 2. Load Data and Analyze Biases

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load data
path = '/kaggle/input/llm-classification-finetuning/'           #for kaggle submission
#path = 'dataset/'                                              #for local test
train_df = pd.read_csv(path+ 'train.csv')
test_df = pd.read_csv(path+ 'test.csv')
submission_df = pd.read_csv(path+ 'sample_submission.csv')

# --- Verbosity Bias Analysis ---
train_df['len_a'] = train_df['response_a'].str.len()
train_df['len_b'] = train_df['response_b'].str.len()
train_df['word_count_a'] = train_df['response_a'].apply(lambda x: len(str(x).split()))
train_df['word_count_b'] = train_df['response_b'].apply(lambda x: len(str(x).split()))
print('--- Verbosity Analysis ---')
print(f"Avg word count for A: {train_df['word_count_a'].mean():.2f}")
print(f"Avg word count for B: {train_df['word_count_b'].mean():.2f}")

# --- Position Bias Analysis ---
model_a_wins = train_df['winner_model_a'].sum()
model_b_wins = train_df['winner_model_b'].sum()
ties = train_df['winner_tie'].sum()
total = len(train_df)
print('--- Position Bias Analysis ---')
print(f'Total samples: {total}')
print(f'Model A wins: {model_a_wins} ({model_a_wins/total:.2%})')
print(f'Model B wins: {model_b_wins} ({model_b_wins/total:.2%})')
print(f'Ties: {ties} ({ties/total:.2%})')


--- Verbosity Analysis ---
Avg word count for A: 204.37
Avg word count for B: 205.18
--- Position Bias Analysis ---
Total samples: 57477
Model A wins: 20064 (34.91%)
Model B wins: 19652 (34.19%)
Ties: 17761 (30.90%)


## 3. Data Preparation

In [3]:
# Prepare original labels
conditions = [train_df['winner_model_a'] == 1, train_df['winner_model_b'] == 1, train_df['winner_tie'] == 1]
choices = [0, 1, 2] # 0: model_a, 1: model_b, 2: tie
train_df['label'] = np.select(conditions, choices, default=-1)
train_df = train_df[train_df['label'] != -1].copy()

# Create combined text field
def create_text(row):
    return f"""prompt: {row['prompt']}

response_a: {row['response_a']}

response_b: {row['response_b']}"""

train_df['text'] = train_df.apply(create_text, axis=1)
test_df['text'] = test_df.apply(create_text, axis=1)

# Split train data for validation
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df['text'], train_df['label'], test_size=0.1, random_state=42, stratify=train_df['label']
)

## 4. Model Fine-tuning with KerasNLP

In [4]:
import keras
import keras_nlp
import tensorflow as tf

peft_config = {"r": 8}
# Load a DeBERTa classifier from the KerasNLP library
classifier = keras_nlp.models.DebertaV3Classifier.from_preset(
    "deberta_v3_small_en",
    num_classes=3,
    peft_config=peft_config 
)

# Compile the model
classifier.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(5e-5),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    jit_compile=True
)

classifier.summary()

# Fit the model
classifier.fit(
    x=train_texts.tolist(),
    y=train_labels.to_numpy(),
    validation_data=(val_texts.tolist(), val_labels.to_numpy()),
    epochs=1,
    batch_size=8
)

2025-10-31 06:56:32.462301: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761893792.672264      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761893792.725671      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1761893806.895199      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Preprocessor: "deberta_v3_text_classifier_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ deberta_v3_tokenizer (DebertaV3Tokenizer)                     │                      Vocab size: 128,001 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "deberta_v3_text_classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ deberta_v3_backbone           │ (None, None, 768)         │     141,304,320 │ padding_mask[0][0],        │
│ (DebertaV3Backbone)           │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ get_item (GetItem)            │ (None, 768)               │               0 │ deberta_v3_backbone[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pooled_dropout (Dropout)      │ (None, 768)               │               0 │ get_item[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pooled_dense (Dense)          │ (None, 768)               │         590,592 │ pooled_dropout[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ classifier_dropout (Dropout)  │ (None, 768)               │               0 │ pooled_dense[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ logits (Dense)                │ (None, 3)                 │           2,307 │ classifier_dropout[0][0]   │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 141,897,219 (541.29 MB)

 Trainable params: 141,897,219 (541.29 MB)

 Non-trainable params: 0 (0.00 B)

I0000 00:00:1761893857.360150      60 service.cc:148] XLA service 0x7c1d64011840 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1761893857.360786      60 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1761893860.726504      60 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1761893878.752139      60 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


6467/6467 ━━━━━━━━━━━━━━━━━━━━ 2092s 315ms/step - loss: 1.1060 - sparse_categorical_accuracy: 0.3495 - val_loss: 1.0936 - val_sparse_categorical_accuracy: 0.3756


In [5]:
from sklearn.isotonic import IsotonicRegression

# Get predictions on the validation set
val_logits = classifier.predict(val_texts.tolist(), batch_size=8)
val_probs = tf.nn.softmax(val_logits).numpy()

# Train a calibrator for each class
calibrators = {}
for i in range(3):
    iso_reg = IsotonicRegression(out_of_bounds='clip')
    y_cal = (val_labels.to_numpy() == i).astype(int)
    iso_reg.fit(val_probs[:, i], y_cal)
    calibrators[i] = iso_reg

print("Calibration models trained.")

719/719 ━━━━━━━━━━━━━━━━━━━━ 74s 97ms/step
Calibration models trained.


## 5. Prediction and Submission

In [6]:
# Predict on the test set
test_logits = classifier.predict(test_df['text'].tolist(), batch_size=8)
test_probs = tf.nn.softmax(test_logits).numpy()

# Apply calibration
calibrated_probs = np.zeros_like(test_probs)
for i in range(3):
    calibrated_probs[:, i] = calibrators[i].predict(test_probs[:, i])

# Normalize probabilities to sum to 1
calibrated_probs_sum = np.sum(calibrated_probs, axis=1, keepdims=True)
# Add a small epsilon to avoid division by zero
normalized_probs = calibrated_probs / (calibrated_probs_sum + 1e-9)

# Create submission file
submission_df['winner_model_a'] = normalized_probs[:, 0]
submission_df['winner_model_b'] = normalized_probs[:, 1]
submission_df['winner_tie'] = normalized_probs[:, 2]

submission_df.to_csv('submission.csv', index=False)

submission_df.head()

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


,id,winner_model_a,winner_model_b,winner_tie
0,136060,0.229521,0.227005,0.543473
1,211333,0.362297,0.375691,0.262012
2,1233961,0.362326,0.375721,0.261953
